In [23]:


# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')




In [24]:
data_frame = pd.read_csv('data.csv')

In [25]:
data_frame.head(475)

,SEMESTRE ACTUAL,ESTRATO,PROMEDIO CARRERA,PROMEDIO SEMESTRE,ESTADO,RESULTADO
0,10,1,2.833,3.50,ACTIVO,0
1,10,3,2.542,0.00,ACTIVO,0
2,6,1,0.000,0.00,INACTIVO,1
3,3,2,3.375,3.70,INACTIVO,1
4,2,3,2.541,3.05,ACTIVO,0
...,...,...,...,...,...,...
470,1,1,0.000,0.00,ACTIVO,0
471,1,1,0.000,0.00,ACTIVO,0
472,1,3,0.000,0.00,ACTIVO,0
473,1,2,0.000,0.00,ACTIVO,0


## Variables de interes 
 semestre actual, Estrato, Promedio de carrera, Promedio de semestre, Estado, Resultado 


# Desarollo 
En esta parte nos interesa explorar los datos y desarrollar el modelo sde regresion logistica, en el estudio de la Desercion estudiantil en el programa de matematicas.

# Exploracion de datos 
vamos a crear los arreglos con los datos de cada una d elas variables de interes 

In [26]:
#array de SEMESTRE ACTUAL
semAct = data_frame['SEMESTRE ACTUAL'].values

#array de ESTRATO
estrato = data_frame['ESTRATO'].values

#array de PROMEDIO CARRERA
promCarr = data_frame['PROMEDIO CARRERA'].values

#array de PROMEDIO SEMESTRE
promSeme = data_frame['PROMEDIO SEMESTRE'].values

#array de RESUTADO
resultados = data_frame['RESULTADO'].values



Se codifica la variable respuesta como 1 si esta inactivo y 0 silo esta, y se identifica cuantas observaciones hay de cada clase.

In [27]:

data_frame['ESTADO'] = np.where(data_frame['ESTADO'] == 'INACTIVO', 1, 0)

print("Número de observaciones por clase")
print(data_frame['ESTADO'].value_counts())
print("")

print("Porcentaje de observaciones por clase")
print(100 * data_frame['ESTADO'].value_counts(normalize=True))



Número de observaciones por clase
0    253
1    222
Name: ESTADO, dtype: int64

Porcentaje de observaciones por clase
0    53.263158
1    46.736842
Name: ESTADO, dtype: float64


El 53,26% de los estudiantes esta activo y el 46.73% estan inactivos, y estos son los datos en lo que tenemos interes. Un modelo de clasificación que sea útil debe de ser capaz de predecir correctamente un porcentaje de observaciones por encima del porcentaje de la clase mayoritaria. En este caso, el umbral de referencia que se tiene que superar es del 53,26%

# Ajuste del modelo

Se ajusta un modelo de regresión logística múltiple con el objetivo de predecir si un estudiante esta  inactivo en función de todas las variables disponibles.

In [28]:
# División de los datos en train y test
# ==============================================================================
X = data_frame.drop(columns = 'ESTADO')
y = data_frame['ESTADO']

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo
X_train = sm.add_constant(X_train, prepend = True)
modelo = sm.Logit(endog = y_train, exog = X_train,)
modelo = modelo.fit()
print(modelo.summary())



         Current function value: 0.000000
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  380
Model:                          Logit   Df Residuals:                      374
Method:                           MLE   Df Model:                            5
Date:                Tue, 05 Dec 2023   Pseudo R-squ.:                   1.000
Time:                        06:49:16   Log-Likelihood:            -8.3869e-05
converged:                      False   LL-Null:                       -262.05
Covariance Type:            nonrobust   LLR p-value:                5.022e-111
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -3.5408   3998.240     -0.001      0.999   -7839.948    7832.867
SEMESTRE ACTUAL      -0.8179    435.635     

# Predicciones


Una vez entrenado el modelo, se pueden obtener predicciones para nuevos datos. Los modelos de statsmodels permiten calcular los intervalos de confianza asociados a cada predicción.

In [30]:
 #Predicciones con intervalo de confianza 
# ==============================================================================
predicciones = modelo.predict(exog = X_train)

# Clasificación predicha
# ==============================================================================
clasificacion = np.where(predicciones<0.5, 0, 1)
clasificacion

array([1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

# Accuracy de test


Se calcula el porcentaje de aciertos que tiene el modelo al predecir las observaciones de test (accuracy).

In [31]:


# Accuracy de test del modelo 
# ==============================================================================
X_test = sm.add_constant(X_test, prepend=True)
predicciones = modelo.predict(exog = X_test)
clasificacion = np.where(predicciones<0.5, 0, 1)
accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = clasificacion,
            normalize = True
           )
print("")
print(f"El accuracy de test es: {100*accuracy}%")




El accuracy de test es: 100.0%


In [32]:


# Matriz de confusión de las predicciones de test
# ==============================================================================
confusion_matrix = pd.crosstab(
    y_test.ravel(),
    clasificacion,
    rownames=['Real'],
    colnames=['Predicción']
)
confusion_matrix



Predicción,0,1
Real,,
0,47,0
1,0,48
